In [1]:
!pip install -qU "OpenDSSDirect.py[extras]" --timeout 60

In [2]:
import opendssdirect as dss
import pandas as pd
import numpy as np

class IEEE33BusSystem:
    def __init__(self):
        # line impedance data (per mile)
        self.line_data = [
            ('sourcebus', '1', 0.0922, 0.0470),
            ('1', '2', 0.4930, 0.2511),
            ('2', '3', 0.3660, 0.1864),
            ('3', '4', 0.3811, 0.1941),
            ('4', '5', 0.8190, 0.4178),
            ('5', '6', 0.1872, 0.0952),
            ('6', '7', 0.7114, 0.3620),
            ('7', '8', 1.0300, 0.5260),
            ('8', '9', 1.0440, 0.5313),
            ('9', '10', 0.1966, 0.1000),
            ('10', '11', 0.3744, 0.1906),
            ('11', '12', 1.4680, 0.7471),
            ('12', '13', 0.5416, 0.2756),
            ('13', '14', 0.5910, 0.3007),
            ('14', '15', 0.7463, 0.3800),
            ('15', '16', 1.2890, 0.6561),
            ('16', '17', 0.7320, 0.3727),
            ('2', '18', 0.1640, 0.0834),
            ('18', '19', 1.5120, 0.7694),
            ('19', '20', 0.4811, 0.2446),
            ('20', '21', 0.8190, 0.4178),
            ('3', '22', 0.4930, 0.2511),
            ('22', '23', 0.6380, 0.3246),
            ('23', '24', 0.4930, 0.2511),
            ('6', '25', 0.1872, 0.0952),
            ('25', '26', 0.7114, 0.3620),
            ('26', '27', 0.4930, 0.2511),
            ('27', '28', 0.3662, 0.1864),
            ('28', '29', 0.6320, 0.3216),
            ('29', '30', 0.3745, 0.1905),
            ('30', '31', 0.3662, 0.1864),
            ('31', '32', 0.4930, 0.2511),
            ('32', '33', 0.5140, 0.2615)  # Added missing connection to bus 33
        ]
        
        # Load data in kW and kVAR
        self.load_data = {
            '2': (100, 60),
            '3': (90, 40),
            '4': (120, 80),
            '5': (60, 30),
            '6': (60, 20),
            '7': (200, 100),
            '8': (200, 100),
            '9': (60, 20),
            '10': (60, 20),
            '11': (45, 30),
            '12': (60, 35),
            '13': (60, 35),
            '14': (120, 80),
            '15': (60, 10),
            '16': (60, 20),
            '17': (60, 20),
            '18': (90, 40),
            '19': (90, 40),
            '20': (90, 40),
            '21': (90, 40),
            '22': (90, 40),
            '23': (90, 50),
            '24': (420, 200),
            '25': (420, 200),
            '26': (60, 25),
            '27': (60, 25),
            '28': (60, 20),
            '29': (120, 80),
            '30': (200, 100),
            '31': (150, 70),
            '32': (210, 100),
            '33': (60, 40)
        }

    def create_circuit(self):
        # Clear any existing circuits
        dss.Basic.ClearAll()

        # Create circuit with detailed source parameters and increased source voltage
        dss.Text.Command(
            'New Circuit.IEEE33Bus ' + 
            'bus1=sourcebus ' + 
            'basekv=12.66 ' + 
            'pu=1.05 ' +
            'phases=3 ' + 
            'MVAsc3=100 ' + 
            'MVAsc1=100'
        )

        # Set voltage bases and calculation method
        dss.Text.Command('Set voltagebases=[12.66]')
        dss.Text.Command('Calcvoltagebases')

        # Add transmission lines with explicit bus connections
        for from_bus, to_bus, r, x in self.line_data:
            line_name = f"Line_{from_bus}_{to_bus}"
            dss.Text.Command(
                f"New Line.{line_name} " + 
                f"bus1={from_bus} " + 
                f"bus2={to_bus} " + 
                f"length=1 " + 
                f"units=mi " + 
                f"phases=3 " + 
                f"r1={r} " + 
                f"x1={x}"
            )

        # Add loads with precise bus naming
        for bus, (p, q) in self.load_data.items():
            dss.Text.Command(
                f"New Load.Load_{bus} " + 
                f"bus1={bus} " + 
                f"phases=3 " + 
                f"kV=12.66 " + 
                f"kW={p} " + 
                f"kvar={q} " + 
                f"model=1 " + 
                f"conn=wye"
            )

        # Adding capacitors at buses with higher loads or at the end of long branches
        capacitor_buses = ['17', '24', '30', '32', '33']
        for bus in capacitor_buses:
            dss.Text.Command(
                f"New Capacitor.Cap_{bus} " +
                f"bus1={bus} " +
                f"phases=3 " +
                f"kvar=300 " +  # Adjust kVAR as needed
                f"kV=12.66 " +
                f"conn=wye"
            )

        # Solution controls
        dss.Text.Command('Set maxiter=200')
        dss.Text.Command('Set tolerance=0.0001')

    def solve_and_analyze(self):
        # Solve the circuit
        try:
            dss.Text.Command('Solve')
            
            # Check solution convergence
            is_converged = dss.Solution.Converged()
            print(f"\nSolution Converged: {is_converged}")

            # Collect bus voltage data
            bus_voltages = []
            bus_names = dss.Circuit.AllBusNames()
            base_voltage = 12660.0  # Base voltage in volts

            for bus in bus_names:
                dss.Circuit.SetActiveBus(bus)
                try:
                    # Get voltage magnitude and angle
                    vmag = dss.Bus.VMagAngle()[0::2]
                    vang = dss.Bus.VMagAngle()[1::2]
                    
                    if vmag and len(vmag) > 0:
                        bus_voltages.append({
                            'Bus': bus,
                            'Voltage (V)': round(vmag[0], 4),
                            # Corrected PU calculation
                            'Voltage (pu)': round(vmag[0] / (base_voltage / np.sqrt(3)), 4),
                            'Angle (deg)': round(vang[0], 4) if vang else None
                        })
                except Exception as e:
                    print(f"Error processing bus {bus}: {e}")

            # Calculate line losses
            line_losses = dss.Circuit.LineLosses()

            # Create DataFrame
            df = pd.DataFrame(bus_voltages)
            
            # Sort DataFrame by bus name numerically (with special handling for 'sourcebus')
            df['SortIndex'] = df['Bus'].apply(lambda x: 0 if x == 'sourcebus' else int(x) if x.isdigit() else float('inf'))
            df = df.sort_values('SortIndex').drop('SortIndex', axis=1)
            
            df.to_csv('IEEE33_Bus_Voltages.csv', index=False)

            print("\nCircuit Analysis Summary:")
            print(f"Total Line Losses: {line_losses[0]:.4f} kW")
            print(f"Number of Buses Analyzed: {len(bus_voltages)}")

            # Add voltage profile analysis
            min_voltage = df['Voltage (pu)'].min()
            max_voltage = df['Voltage (pu)'].max()
            avg_voltage = df['Voltage (pu)'].mean()
            
            print(f"\nVoltage Profile Analysis:")
            print(f"Minimum Voltage: {min_voltage:.4f} pu")
            print(f"Maximum Voltage: {max_voltage:.4f} pu")
            print(f"Average Voltage: {avg_voltage:.4f} pu")
            
            return df, line_losses[0]

        except Exception as solve_err:
            print("Circuit Solve Error:", str(solve_err))
            return None, None

# Run the simulation
def main():
    # Create simulation instance
    sim = IEEE33BusSystem()
    
    # Create circuit
    sim.create_circuit()
    
    # Solve and analyze
    results, losses = sim.solve_and_analyze()

    if results is not None:
        print("\nVoltage Summary:")
        print(results)

In [3]:
if __name__ == "__main__":
    main()


Solution Converged: True

Circuit Analysis Summary:
Total Line Losses: 170.9664 kW
Number of Buses Analyzed: 34

Voltage Profile Analysis:
Minimum Voltage: 0.9700 pu
Maximum Voltage: 1.0362 pu
Average Voltage: 0.9935 pu

Voltage Summary:
          Bus  Voltage (V)  Voltage (pu)  Angle (deg)
0   sourcebus    7573.9818        1.0362      -1.9296
1           1    7557.3132        1.0339      -1.9771
2           2    7468.2757        1.0218      -2.2343
3           3    7411.6318        1.0140      -2.4271
4           4    7364.8131        1.0076      -2.5869
5           5    7270.3525        0.9947      -2.9430
6           6    7249.4472        0.9918      -3.0254
7           7    7213.4774        0.9869      -3.1074
8           8    7173.3523        0.9814      -3.2273
9           9    7144.8868        0.9775      -3.3486
10         10    7140.1725        0.9769      -3.3708
11         11    7132.4226        0.9758      -3.4116
12         12    7106.1989        0.9722      -3.5760
13   

In [5]:
import os

directory_path = "./"

# List all files in the directory
try:
    files = os.listdir(directory_path)
    print("Files in directory:")
    for file_name in files:
        print(file_name)
except FileNotFoundError:
    print(f"The directory '{directory_path}' does not exist.")

Files in directory:
augmentation.ipynb
BusCoords.dss
BusVoltageBases.dss
df.ipynb
GrowthShape.dss
IEEE33_Bus_Voltages.csv
Line.dss
Load.dss
LoadShape.dss
Master.dss
monte_carlo_load_data.csv
Spectrum.dss
TCC_Curve.dss
voltage_profile.png
Vsource.dss
